https://docs.microsoft.com/en-us/learn/paths/build-ai-solutions-with-azure-ml-service/

https://thenewstack.io/build-and-deploy-a-machine-learning-model-with-azure-ml-service/

In [1]:
# !pip install statsmodels


In [1]:
import datetime
import numpy as np
import pandas as pd
pd.set_option('display.precision',3)
from sklearn.externals import joblib
 
import azureml.core
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core import Experiment
from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage
from azureml.core.webservice import AciWebservice
from azureml.core.conda_dependencies import CondaDependencies

from azureml.core import Workspace, Experiment, Run

import math, random, pickle

import matplotlib.pyplot as plt
from statsmodels.tsa.holtwinters import ExponentialSmoothing

from sklearn.metrics import mean_squared_error
from math import sqrt

import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
# ws = Workspace.from_config(aml)

In [3]:
# ws = Workspace.create(name='aml',
#                       subscription_id='xxxxxxxxxxxxxxxxxxxxxxxxxx', 
#                       resource_group='ts_test',
#                       create_resource_group = True,
#                       location='southeastasia'
#                      )


ws = Workspace.get(name='aml',
                      subscription_id='xxxxxxxxxxxxxxxxxxxxxxxxxx', 
                      resource_group='ts_test',
#                       create_resource_group = True,
#                       location='southeastasia'
                     )

In [4]:
# from azureml.core import Workspace, Experiment, Run
# import math, random, pickle, json


exp = Experiment(workspace = ws, name='ts')
run = exp.start_logging()                   
run.log("Experiment start time", str(datetime.datetime.now()))

In [5]:
###############SQL Alchemy for data uploading#################
from sqlalchemy import create_engine 
import urllib
server = 'TE\SQL2016'
database = 'ABC'
username = 'sa'
password = 'sa'
#driver='{ODBC Driver 17 for SQL Server}'
#Enabled = 'Enabled'
params = urllib.parse.quote_plus(
'DRIVER={ODBC Driver 17 for SQL Server};'+ 
'SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password) 

engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params) 

###Connection checking
connected = pd.io.sql._is_sqlalchemy_connectable(engine)
print (connected)

True


In [1]:
df_is = pd.read_csv('D:\.csv', parse_dates=['Date'],dayfirst=True)
is_df = df_is.sort_values(['year', 'month'], ascending=[True, True])

is_df.head()

In [7]:
is_df['Amount'] = is_df['Amount'].abs()
is_df = is_df.loc[(is_df['Session Code'].isin(['Operating Expenses'])) & (is_df['Book'].isin(['Accrual']))]
is_df = is_df.groupby(['pro Name','Date']).agg({'Amount':sum}).reset_index()

df_exp = is_df #.loc[is_df['Amount'] != 0]

In [8]:
len(df_exp.loc[df_exp['Amount'] < 0])

0

In [9]:
df_exp.set_index('Date',inplace=True)

In [2]:
# df_exp.head()

In [3]:
tedf = df_exp.loc[(df_exp['pro Name'] == 'xyz')]
tedf['Amount'].iloc[tedf.index == '2019-11-01'] = 0
tedf = tedf['Amount']
tedf.tail()

In [12]:
def exp_smoothing_configs(seasonal=[None]):
    models = list()
    # define config lists
    t_params = ['add', 'mul', None]
    d_params = [True, False]
    s_params = ['add', 'mul', None]
    p_params = seasonal
    b_params = [True, False]
    r_params = [True, False]
    # create config instances
    for t in t_params:
        for d in d_params:
            for s in s_params:
                for p in p_params:
                    for b in b_params:
                        for r in r_params:
                            cfg = [t,d,s,p,b,r]
                            models.append(cfg)
    return models

In [13]:
cfg_list = exp_smoothing_configs(seasonal=[12])
# cfg_list = exp_smoothing_configs(seasonal=[0,6,12])
len(cfg_list)

72

In [14]:
def exp_smoothing_forecast(data, config):
    t,d,s,p,b,r = config
    train = data[:'2018'].copy()
    test = data['2019-01-01':'2019-10-01'].copy()
    # define model
    if (t == None):
        model = ExponentialSmoothing(train, trend=t, seasonal=s, seasonal_periods=p)
    else:
        model = ExponentialSmoothing(train, trend=t, damped=d, seasonal=s, seasonal_periods=p)
    # fit model
    model_fit = model.fit(optimized=True, use_boxcox=b, remove_bias=r)
    # make one step forecast
    y_forecast = model_fit.forecast(10)
    
    rmse = np.sqrt(mean_squared_error(test,y_forecast))
    return rmse

In [4]:
# set(df_exp['pro Name'])

In [51]:
edf = df_exp['Amount'].loc[(df_exp['pro Name'] == 'abc')]
edf = edf.groupby(['Date']).agg({'Amount':sum})
edf = edf['Amount']
ts = edf[:'2018'].copy()
ts_v = edf['2019-01-01':'2019-10-01'].copy()
ts1  = edf['2014-01-01':'2019-10-01'].copy()
#     ind = edf.index[-12:]  # this will select last 12 months' indexes

print("Holt's Winter Model")

best_RMSE = np.inf
best_config = []
t1 = d1 = s1 = p1 = b1 = r1 = ''
for j in range(len(cfg_list)):
    print(j)
    try:
        cg = cfg_list[j]
        print(cg)
        rmse = exp_smoothing_forecast(edf, cg)
        print(rmse)
        if rmse < best_RMSE:
            best_RMSE = rmse
            best_config = cfg_list[j]
    except:
        print("Error")
        continue

print(best_RMSE, best_config)

Holt's Winter Model
0
['add', True, 'add', 12, True, True]
Error
1
['add', True, 'add', 12, True, False]
Error
2
['add', True, 'add', 12, False, True]
305470.7094210574
3
['add', True, 'add', 12, False, False]
292545.1927775726
4
['add', True, 'mul', 12, True, True]
Error
5
['add', True, 'mul', 12, True, False]
Error
6
['add', True, 'mul', 12, False, True]
316944.8660490859
7
['add', True, 'mul', 12, False, False]
304819.502671156
8
['add', True, None, 12, True, True]
Error
9
['add', True, None, 12, True, False]
Error
10
['add', True, None, 12, False, True]
182679.5297555348
11
['add', True, None, 12, False, False]
183746.63005922126
12
['add', False, 'add', 12, True, True]
Error
13
['add', False, 'add', 12, True, False]
Error
14
['add', False, 'add', 12, False, True]
305470.7094210574
15
['add', False, 'add', 12, False, False]
292545.1927775726
16
['add', False, 'mul', 12, True, True]
Error
17
['add', False, 'mul', 12, True, False]
Error
18
['add', False, 'mul', 12, False, True]
31693

In [52]:
t1,d1,s1,p1,b1,r1 = best_config

if t1 == None:
    hw_model1 = ExponentialSmoothing(ts1, trend=t1, seasonal=s1, seasonal_periods=p1) #ts
else:
    hw_model1 = ExponentialSmoothing(ts1, trend=t1, seasonal=s1, seasonal_periods=p1, damped=d1) #ts
fit2 = hw_model1.fit(optimized=True, use_boxcox=b1, remove_bias=r1)

pred_HW = fit2.predict(start=pd.to_datetime('2019-01-01'), end = pd.to_datetime('2019-10-01'))
# pred_HW = fit2.predict(start=pd.to_datetime('2019-01-01'), end = pd.to_datetime('2019-12-01'))
pred_HW = pd.Series(data=pred_HW)
df_exp_pred = pd.concat([df_exp.loc[(df_exp['pro Name'] == 'abc')], pred_HW.rename('pred_HW')], axis=1)
# pred_data.append(df_exp_pred)
#     print(model_eval(y_truth, pred_6_HW))
print('-*-'*20)

# 143654.42930094403 [None, True, 'add', 12, False, True]
# 162996.80021279448 [None, True, 'add', 12, False, True]

-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-


In [53]:
# fit2.forecast(3)

In [54]:
float(1.788323e+06)

# df_exp_pred['pred_HW']

1788323.0

In [49]:
def model_eval(y, predictions):
    
    # Import library for metrics
    from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

    # Mean absolute error (MAE)
    mae = mean_absolute_error(y, predictions)

    # Mean squared error (MSE)
    mse = mean_squared_error(y, predictions)

    
    # SMAPE is an alternative for MAPE when there are zeros in the testing data. It
    # scales the absolute percentage by the sum of forecast and observed values
    SMAPE = np.mean(np.abs((y - predictions) / ((y + predictions)/2))) * 100


    # Calculate the Root Mean Squared Error
    rmse = np.sqrt(mean_squared_error(y, predictions))
    
    # Calculate the Mean Absolute Percentage Error
    #     y, predictions = check_array(y, predictions)
    MAPE = np.mean(np.abs((y - predictions) / y)) * 100
    
    #     mean_forecast_error
    mfe = np.mean(y - predictions)
    
    # NMSE normalizes the obtained MSE after dividing it by the test variance. It
    # is a balanced error measure and is very effective in judging forecast
    # accuracy of a model.

    #     normalised_mean_squared_error
    NMSE = mse / (np.sum((y - np.mean(y)) ** 2)/(len(y)-1))

    
    #theil_u_statistic
    # It is a normalized measure of total forecast error.
    error = y - predictions
    mfe = np.sqrt(np.mean(predictions**2))
    mse = np.sqrt(np.mean(y**2))
    rmse = np.sqrt(np.mean(error**2))
    theil_u_statistic =  rmse / (mfe*mse)


    #     mean_absolute_scaled_error
    # This evaluation metric is used to over come some of the problems of MAPE and
    # is used to measure if the forecasting model is better than the naive model or
    # not.
    

    # Print metrics
    print('Mean Absolute Error:', round(mae, 3))
    print('Mean Squared Error:', round(mse, 3))
    print('Root Mean Squared Error:', round(rmse, 3))
    print('Mean absolute percentage error:', round(MAPE, 3))
    print('Scaled Mean absolute percentage error:', round(SMAPE, 3))
    print('Mean frequency error:', round(mfe, 3))
    print('Normalised mean squared error:', round(NMSE, 3))
    print('Theil_u_statistic:', round(theil_u_statistic, 3))

In [50]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
mean_squared_error(ts_v, pred_HW)
model_eval(ts_v, pred_HW)  

Mean Absolute Error: 26694.129
Mean Squared Error: 152032.516
Root Mean Squared Error: 34972.01
Mean absolute percentage error: 21.709
Scaled Mean absolute percentage error: 18.707
Mean frequency error: 152477.575
Normalised mean squared error: 0.843
Theil_u_statistic: 0.0


In [55]:
filename = 'outputs/bdgt_ts_model.pkl'
joblib.dump(fit2, filename)

['outputs/bdgt_ts_model.pkl']

In [56]:
# model.slope

In [57]:
# run.log('Intercept :', model.sse)
# run.log('Slope :', model.slope[0])
 
run.log("Experiment end time", str(datetime.datetime.now()))
run.complete()

In [58]:
model = Model.register(model_path = "outputs/bdgt_ts_model.pkl",
                       model_name = "bdgt_ts_model",
                       tags = {"key": "1"},
                       description = "Budget Forecast",
                       workspace = ws)

Registering model bdgt_ts_model


In [59]:
filename = 'outputs/bdgt_ts_model.pkl'
loaded_model = joblib.load(filename)
data = [['2019-01-01', '2019-12-01']]
y = loaded_model.forecast(6)#predict(start = data[0][0], end = data[0][1])
print(list(y.index))

[Timestamp('2019-11-01 00:00:00', freq='MS'), Timestamp('2019-12-01 00:00:00', freq='MS'), Timestamp('2020-01-01 00:00:00', freq='MS'), Timestamp('2020-02-01 00:00:00', freq='MS'), Timestamp('2020-03-01 00:00:00', freq='MS'), Timestamp('2020-04-01 00:00:00', freq='MS')]


In [11]:
# print(y.index.values)
d = list(y.index.strftime('%Y-%m-%d').values)
a = list(y.values)
dict(zip(d,a))

In [5]:
print(run.get_portal_url())

In [62]:
tsenv = CondaDependencies()
tsenv.add_conda_package("scikit-learn")
tsenv.add_conda_package("matplotlib")
tsenv.add_conda_package("statsmodels")

In [63]:
with open("tsenv.yml","w") as f:
    f.write(tsenv.serialize_to_string())
with open("tsenv.yml","r") as f:
    print(f.read())

# Conda environment specification. The dependencies defined in this file will

# be automatically provisioned for runs with userManagedDependencies=False.


# Details about the Conda environment file format:

# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually


name: project_environment
dependencies:
  # The python interpreter version.

  # Currently Azure ML only supports 3.5.2 and later.

- python=3.6.2

- pip:
    # Required packages for AzureML execution, history, and data preparation.

  - azureml-defaults

- scikit-learn
- matplotlib
- statsmodels
channels:
- conda-forge



In [64]:
import json
# np.array(json.loads([['42']]))

json.dumps(y.tolist())

'[1574552.8215747683, 1574539.64301587, 1574538.2557991436, 1574538.1097763304, 1574538.094405508, 1574538.0927875265]'

In [65]:
%%writefile score.py
 
import json
from sklearn.externals import joblib
import os
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.holtwinters import ExponentialSmoothing
 
from azureml.core.model import Model
 
def init():
    global model
    # retrieve the path to the model file using the model name
    model_path = Model.get_model_path('bdgt_ts_model')
    model = joblib.load(model_path)
 
def run(raw_data):
    data = np.array(json.loads(raw_data)['data'])
    print(data)
    # make prediction
    predi = model.forecast(data)
    d = list(predi.index.strftime('%Y-%m-%d').values)
    a = list(predi.values)
    dd = dict(zip(d,a))
    return json.dumps(dd)

Overwriting score.py


In [66]:
%%time
 
image_config = ContainerImage.image_configuration(execution_script="score.py", 
                                                  runtime="python", 
                                                  conda_file="tsenv.yml")

Wall time: 999 µs


In [67]:
aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "aml",  "method" : "ts"}, 
                                               description='Budget Forecast')

In [70]:
service = Webservice.deploy_from_model(workspace = ws,
                                       name = 'budget-ts-forecast-125-park2',
                                       deployment_config = aciconfig,
                                       models = [model],
                                       image_config = image_config)

Creating image
Running.....................................................................
Succeeded
Image creation operation finished for image budget-ts-forecast-125-park2:1, operation "Succeeded"


In [71]:
service.wait_for_deployment(show_output=True)

Running......................
SucceededACI service creation operation finished, operation "Succeeded"


In [72]:
# print(service.get_logs())

In [8]:
print(service.scoring_uri)

In [99]:
# !curl -X POST -H 'Content-Type':'application/json' -d '{data:[['1960-01-01', '1960-12-01']]}' http://xxxxxxxxxxxxxxxxxxxx.southeastasia.azurecontainer.io/score

In [ ]:
# Run in the browser

# http://xxxxxxxxxxxxxxxxxxxxxxxxxxxx.southeastasia.azurecontainer.io/score?data=[[45]]


# Output: 
    
# "[185924.79674796748]"

In [116]:
# ws.delete()

# =============================================================

In [ ]:
  
#!flask/bin/python
from flask import Flask, jsonify
from sklearn.externals import joblib


filename = 'model/sal_model.pkl'

app = Flask(__name__)

@app.route('/')
def index():
    return "Stackoverflow Salary Predictor"

@app.route('/sal/<int:x>', methods=['GET'])
def predict(x):
    loaded_model=joblib.load(filename)
    y=loaded_model.predict([[x]])[0]
    sal=jsonify({'salary': round(y,2)})
    return sal

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=80)

In [1]:
from azureml.core import Workspace, Experiment, Run
import math, random, pickle

In [2]:
import datetime
import dateutil.relativedelta

d = datetime.datetime.strptime("2013-05-01", "%Y-%m-%d")
d2 = d - dateutil.relativedelta.relativedelta(months=1)
print(d2)

2013-04-01 00:00:00
